In [1]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gpu_options = tf.GPUOptions(allow_growth=True)
sess_config = tf.ConfigProto(gpu_options=gpu_options)
tf.keras.backend.set_session(tf.Session(config=sess_config))

In [2]:
import os
import glob
#import tqdm
import numpy as np
#import pandas as pd
import multiprocessing
import pydicom as dicom
import nibabel as nib
#from keras import utils as kutils
from matplotlib import pyplot as plt

def load_image(label_fpath, transpose=False):
    #encode_name = label_fpath[-39: -7]
    label_data = nib.load(label_fpath)
    label_array = label_data.get_fdata()
    if transpose:
        label_array = np.transpose(label_array, axes=(2, 1, 0))
    return  label_array#, encode_name

In [3]:
import pandas as pd
info = pd.read_csv('/data2/pancreas/Nifti_data/data_list.csv')

info['total'] = [1]*len(info)
pd.DataFrame(info.groupby(['cut', 'type']).sum().total)

total
cut   type          
fine  healthy    102
      tumor       44
thick healthy     82
      tumor      197

In [7]:
pd.DataFrame(info.groupby(['resection', 'stage']).sum().total)

total
resection stage       
0.0       I          1
          IIA        3
          IIB        4
          III       22
          IV        65
          X          1
1.0       IB         4
          II         1
          IIA       19
          IIB       44
          III       10
          IV         8

In [6]:
info.head()

,patient_id,case_id,exam_date,type,cut,200_list,diff_patient_list,add_date,resection,stage,size,total
0,26,NP1,20180101.0,healthy,fine,validation,True,20190210,NaN,NaN,NaN,1
1,236,AD1,20170928.0,healthy,fine,validation,True,20190210,NaN,NaN,NaN,1
2,216,AD10,20160920.0,healthy,fine,train,True,20190210,NaN,NaN,NaN,1
3,185,AD100,20140102.0,healthy,fine,train,True,20190210,NaN,NaN,NaN,1
4,164,AD101,20140204.0,healthy,fine,test,True,20190210,NaN,NaN,NaN,1


In [8]:
import pandas as pd
info = pd.read_csv('/data2/pancreas/Nifti_data/data_list.csv')
image_path = "/data2/pancreas/Nifti_data/image"
label_path = "/data2/pancreas/Nifti_data/label"


resec = np.load("resec.npy")
y = np.load("resec_y.npy")
shape_tumor = np.load("shape_tumor.npy")

In [9]:
stage = []
size = []
cut = []
for name in resec:
    stage.append(info.iloc[np.where(info.case_id == name)].stage.values[0])
    size.append(info.iloc[np.where(info.case_id == name)]['size'].values[0])
    cut.append(info.iloc[np.where(info.case_id == name)].cut.values[0])

In [10]:

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = stage
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# print(inverted)



['IIA' 'IIB' 'III' 'IIB' 'IIB' 'IV' 'IIB' 'III' 'IIB' 'IV' 'IV' 'IV' 'IIB'
 'IV' 'IIA' 'IIB' 'III' 'IIB' 'III' 'IV' 'IIB' 'IV' 'IV' 'IV' 'IIB' 'IIA'
 'IB' 'IV' 'IV' 'IIB' 'IIB' 'III' 'IIB' 'III' 'IIA' 'IIA' 'IV' 'IV' 'IV'
 'IV' 'IV' 'IV' 'III' 'IIB' 'IIB' 'IIB' 'IIA' 'IIB' 'IIA' 'IIA' 'IIB'
 'IIB' 'IV' 'IV' 'IV' 'IV' 'IB' 'IV' 'IIB' 'IIB' 'III' 'IV' 'IIB' 'III'
 'IIB' 'IV' 'IIB' 'IIA' 'IIB' 'IV' 'X' 'IV' 'IIB' 'III' 'IV' 'IV' 'III'
 'IV' 'III' 'IV' 'III' 'IV' 'IIB' 'IV' 'IV' 'IIA' 'IV' 'III' 'III' 'IIA'
 'IIA' 'IIA' 'IIB' 'IV' 'III' 'IV' 'IIA' 'IIB' 'IIB' 'IIA' 'IIB' 'IIB'
 'IV' 'IIB' 'IV' 'IV' 'IV' 'III' 'IV' 'III' 'IIA' 'IIB' 'IV' 'IV' 'I' 'IV'
 'IIB' 'IIB' 'IIB' 'IV' 'IV' 'III' 'IV' 'III' 'III' 'IV' 'IIB' 'IIA' 'IV'
 'III' 'IV' 'IIB' 'IV' 'IIA' 'IIB' 'IB' 'III' 'III' 'IV' 'IV' 'IIB' 'IV'
 'IV' 'IV' 'IV' 'IV' 'III' 'IV' 'IIA' 'IIB' 'III' 'III' 'IIB' 'IV' 'IV'
 'IV' 'IV' 'IV' 'IIA' 'IIB' 'IV' 'IIB' 'IIB' 'IV' 'III' 'IIA' 'IIB' 'IIA'
 'IB' 'II' 'IV' 'IV' 'III' 'IV' 'III' 'III' 'IV' 'IV

/opt/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
import os
tumor_mask_npz_folder = './npz/tumor_mask_images/'

In [12]:
#New
def extract_cube(r, img):#cube 210, 220, 90 ;; ranges = [xmin, xmax, ymin, ymax, zmin, zmax]
    img_tmp = np.zeros((210, 220, 90))
    #print(img.shape, r)
    img_tmp[:int(r[1]-r[0]), :int(r[3]-r[2]), :int(r[5]-r[4])] = img[int(r[0]):int(r[1]), int(r[2]):int(r[3]), int(r[4]):int(r[5])]
    return img_tmp


def masking(label, img):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if label[i, j] == 0 or label[i, j] == 1:
                img[i, j] = 0
    return img

def clip_cube(img, i):#Input((70, 55, 30, 1)) ; the i-th cube   3*4*3
    j = i%3
    k = i%4
    l = i%3
    #print(img[j*60:(j+1)*60, k*40:(1+k)*40, l*30:(l+1)*30].shape)
    return img[j*70:(j+1)*70, k*55:(1+k)*55, l*30:(l+1)*30]

import pandas as pd
map_df = pd.DataFrame(data={'resec_name': resec, 'shape': np.array(range(182)), 'target':y.reshape(182)})

In [13]:
from keras.models import Model#, load_model
from keras import layers as klayers
from keras.optimizers import Adam
from keras import utils as kutils
from keras import backend as K
from keras.callbacks import ModelCheckpoint



from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
#from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
#import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py

#init_notebook_mode(connected=True)
%matplotlib inline


from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, concatenate,  MaxPool2D
# from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Lambda
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv1D
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector, Reshape
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline



Using TensorFlow backend.


In [23]:
from keras.utils import to_categorical

class resec_Generator1(kutils.Sequence):
    'Generates data for Keras'
    def __init__(self, mapping_df, batch_size, shuffle=True):
        'Initialization'
        self.mapping_df = mapping_df
        self.data_num   = mapping_df.shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.data_num / self.batch_size))


    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        #print("enter0")
        batch_mapping_df = \
            self.mapping_df.iloc[index*self.batch_size: (index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(batch_mapping_df)
        return X, y


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            self.mapping_df = self.mapping_df.sample(frac=1).reset_index(drop=True)
            
    def __data_generation(self, batch_mapping_df):
        'Generates data containing batch_size samples' 
        # Initialization
        X = [np.zeros((self.batch_size, 70, 55, 30, 1)) for i in range(36)]
        x1 = np.zeros((self.batch_size, 1))
        x2 = np.zeros((self.batch_size, 8))
        y = np.zeros((self.batch_size))

        # Generate data
        cnt = 0
        
        for i, row in batch_mapping_df.iterrows():
            #img = extract_cube(shape_tumor[row['shape']], load_image(os.path.join(image_path, "IM_"+row['resec_name']+".nii.gz")))
            encode = row['resec_name']
            volume_image = load_image(os.path.join(image_path, "IM_"+encode+".nii.gz"))
            label = load_image(os.path.join(label_path, "LB_"+encode+".nii.gz"))
            volume_image1 = extract_cube(shape_tumor[row['shape'], :], volume_image) 
            label1 = extract_cube(shape_tumor[row['shape'], :], label) 
            num_slice = volume_image1.shape[2]
            for _z in range(0, num_slice):
                label11 = ndimage.binary_dilation(label1[:, :, _z]).astype(label1[:, :, _z].dtype)
                volume_image1[..., _z] = masking(label11, volume_image1[..., _z])
                del label11
            for j in range(36):
                
                X[j][cnt, :, :, :, 0] = clip_cube(volume_image1, j)
            
            
            x1[cnt, :] = size[row['shape']]
            x2[cnt, :] = onehot_encoded[row['shape'], :]
            del volume_image1, label1, volume_image, label
           
            y[cnt] = row['target']


            cnt += 1
            X.append(x1)
            X.append(x2)
        return X, y

# for name in resec:
#     img = load_image(os.path.join(image_path, "IM_"+name+".nii.gz"))

In [29]:
del model

In [30]:
from keras import metrics
xs = []
input_set = []
for i in range(36):
    tmp_x = Input((70, 55, 30, 1))
    input_set.append(tmp_x)
    x = Conv3D(filters=32, dilation_rate=1, kernel_size=(3, 3, 3), kernel_initializer=keras.initializers.glorot_normal(seed=None), activation='selu')(tmp_x)
    #x = BatchNormalization()(x)

    x = MaxPool3D()(x)

    x = Conv3D(filters=16, dilation_rate=1, kernel_size=(3, 3, 3),  activation='relu')(x)
    #x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = MaxPool3D()(x)

    x = Conv3D(filters=4, dilation_rate=1, kernel_size=(3, 3, 3), activation='relu')(x)
    #x = BatchNormalization()(x)
  
    x = MaxPool3D()(x)
    
    
    
    xs.append(x)
    
x = xs[0]
for i in range(1, 36):
    x = concatenate([x, xs[i]], axis = 4)
    

x = MaxPool3D()(x)
x = Conv3D(filters=32, dilation_rate=1, kernel_size=(1, 1, 1), kernel_initializer=keras.initializers.glorot_normal(seed=None), activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

x = Conv3D(filters=4, dilation_rate=1, kernel_size=(1, 1, 1), activation='relu')(x)
x = BatchNormalization()(x)



x = Flatten()(x)
x = Dense(units=500, kernel_initializer='normal', activation='elu')(x)
x = Dense(units=100, kernel_initializer='normal', activation='relu')(x)
x = Dense(units=1, kernel_initializer='normal', activation='sigmoid')(x)


tmp_a = Input((1,))
a = Dense(5, activation='elu')(tmp_a)
a = Dense(2, activation='relu')(a)
a = Dense(1,  activation='sigmoid')(a)

tmp_b = Input((8,))
b = Dense(10, activation='selu')(tmp_b)
b = Dense(20, activation='selu')(b)
b = Dense(10, activation='elu')(b)
b = Dense(5, activation='relu' )(b)
b = Dense(1, activation='sigmoid')(b)

x = concatenate([x, a, b], axis = 1)
x = Dense(1, activation='sigmoid')(x)


input_set.append(tmp_a)
input_set.append(tmp_b)


model = Model(inputs=input_set, outputs=x)
adam_lr = 0.0002
adam_beta_1 = 0.5
model.compile(optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
              loss='binary_crossentropy', metrics = [metrics.binary_accuracy])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_153 (InputLayer)          (None, 70, 55, 30, 1 0                                            
__________________________________________________________________________________________________
input_154 (InputLayer)          (None, 70, 55, 30, 1 0                                            
__________________________________________________________________________________________________
conv3d_441 (Conv3D)             (None, 68, 53, 28, 3 896         input_153[0][0]                  
__________________________________________________________________________________________________
conv3d_444 (Conv3D)             (None, 68, 53, 28, 3 896         input_154[0][0]                  
__________________________________________________________________________________________________
input_155 

In [31]:
batch_size = 1
v_generator = resec_Generator1(map_df[:-50], batch_size=batch_size)
validation_generator = resec_Generator1(map_df[-50:-30], batch_size=batch_size)

In [ ]:
from scipy import ndimage
history = model.fit_generator(v_generator,
                              epochs=100, validation_data=validation_generator)

Epoch 1/100
  5/132 [>.............................] - ETA: 11:24 - loss: 0.6684 - binary_accuracy: 0.6000

In [27]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

NameError: name 'history' is not defined